<a href="https://colab.research.google.com/github/OkitegamiKyoko/Spectrogram-Generator/blob/main/Generador_de_espetrogramas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-io matplotlib pydub

In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/My Drive/data')

In [ ]:
AUDIO_FILE = os.path.join('data', 'prueba.wav')

# Stereo a Mono
Esta función transforma el archivo wav de estereo a mono, por lo que unicamente puede recibir archivos de estereo.

In [ ]:
sample_rate2 = 0
def load_wav_16k_mono(filename):
    # Load encoded wav file - Only one
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    sample_rate2 = sample_rate
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    print(int(sample_rate))
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=20000)
    return wav

In [ ]:
wave = load_wav_16k_mono(AUDIO_FILE)

# Impresion de la forma de onda

In [ ]:
plt.figure(figsize=(30,20))
plt.plot(wave)
plt.show()

Para archivos de 3 seg

In [ ]:
def preprocess(file_path):
    wav = load_wav_16k_mono(file_path)
    wav = wav[:480000]
    zero_padding = tf.zeros([480000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [ ]:
spectrogram = preprocess(AUDIO_FILE)

In [ ]:
#plt.imshow(tf.math.log(spectrogram + 1e-6).numpy(), cmap='viridis', origin='lower', aspect='auto')
plt.imshow(tf.transpose(tf.math.log(spectrogram + 1e-6).numpy())[0], cmap='viridis', origin='lower', aspect='auto')
plt.colorbar(format='%+2.0f dB')
plt.show()

# Filtros
Para fortuna nuestra, la mayoría de los cantos de las aves están en un rango de 2 000 a 7 000 Hz, con un óptimo entre 1 000 y 5 000 Hz (User, s. f.).
# Pasa Banda
Frecuencia de corte entre 1kHz y 7khz

# Referencia
User, S. (s. f.). Estructura y evolución de las vocalizaciones de las aves - Revista Ciencias. https://www.revistacienciasunam.com/en/149-revistas/revista-ciencias-109-110/1238-estructura-y-evoluci%C3%B3n-de-las-vocalizaciones-de-las-aves.html

In [ ]:
import scipy.ndimage
import numpy as np
import scipy.signal

def filtro_pasa_banda(signal, low_cutoff, high_cutoff, sample_rate):
    nyquist = 0.5 * sample_rate
    low = low_cutoff / nyquist
    high = high_cutoff / nyquist
    b, a = scipy.signal.butter(10, [low, high], btype='band', analog=False)
    filtered_signal = scipy.signal.filtfilt(b, a, signal)
    return filtered_signal

# Generar espectrograma
Procesa el archivo wav para generar su resectivo espectrograma, se recomienda que los archivos sean de la misma duración para evitar problemas de tamaño y nitidez de la imagen resultante

In [ ]:
def preprocess(file_path):
    cutoff_frequency_low=1000
    cutoff_frequency_high=7000
    # Cargar el archivo de audio de música y convertirlo a una forma de onda
    wav = load_wav_16k_mono(file_path)
    # Calcular el espectrograma utilizando la Transformada de Fourier de Tiempo Corto (STFT)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    # Calcular el valor absoluto del espectrograma para obtener magnitudes de frecuencia
    spectrogram = tf.abs(spectrogram)
    # Aplicar el filtro pasa altas a cada fila del espectrograma
    # spectrogram = tf.constant([filtro_pasa_banda(row, cutoff_frequency_low, cutoff_frequency_high, 41000) for row in spectrogram.numpy()])
    # spectrogram = tf.expand_dims(spectrogram, axis=2)

    # Aplicar el filtro pasa banda a cada fila del espectrograma
    # spectrogram = np.apply_along_axis(lambda row: filtro_pasa_banda(row, cutoff_frequency_low, cutoff_frequency_high, 41000), axis=1, arr=spectrogram.numpy())

    # Calcular el espectrograma de Mel
    '''
    mel_spectrogram = tf.signal.linear_to_mel_weight_matrix(
        num_mel_bins=128,  # Número de bandas de Mel
        num_spectrogram_bins=spectrogram.shape[-1],
        sample_rate=16000,  # Frecuencia de muestreo del audio
        lower_edge_hertz=0,  # Frecuencia mínima de Mel
        upper_edge_hertz=8000)  # Frecuencia máxima de Mel

    mel_spectrogram = tf.matmul(tf.square(spectrogram), mel_spectrogram)

    # Logaritmo para obtener la escala logarítmica de Mel
    mel_spectrogram = tf.math.log(mel_spectrogram + 1e-6)
    '''

    # Expandir las dimensiones del espectrograma para agregar una dimensión adicional
    spectrogram = tf.expand_dims(spectrogram, axis=2)

    return spectrogram

    # Expandir las dimensiones del espectrograma para agregar una dimensión adicional
    # spectrogram = tf.expand_dims(spectrogram, axis=2)
    # Devolver el espectrograma preprocesado
    # return spectrogram

In [ ]:
# Reemplaza 'tu_ruta_de_audio' con la ruta real de tu archivo de audio

import numpy as np

def gen_spect(Path,num):
  spectrogram = preprocess(Path)
  # Rotar los datos del espectrograma
  # rotated_spectrogram = np.rot90(tf.math.log(spectrogram + 1e-6).numpy(), k=1)
  # Configurar el tamaño de la figura sin márgenes
  fig, ax = plt.subplots(figsize=(15, 10), tight_layout=True)
  ax.imshow(tf.transpose(tf.math.log(spectrogram + 1e-6).numpy())[0], cmap='gray_r', origin='lower', aspect='auto')
  # Ocultar etiquetas de eje
  # ax.set_xticks([])
  # ax.set_yticks([])
  ax.axis('off')
  plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
  # plt.xlabel('Frecuencia bin')
  # plt.ylabel('Tiempo')
  # plt.title('Espectrograma de Audio Completo')
  # plt.colorbar(format='%+2.0f dB')
  # Guarda la imagen del spectrograma
  fig.savefig('/content/espectrograma/espectrograma'+str(num)+'.png', bbox_inches='tight', pad_inches=0)
  plt.show()

Busca todos los audios dentro de una carpeta

In [ ]:
import glob

# Define el patrón del nombre de archivo que estás buscando
pattern = '/content/drive/My Drive/data/output_*.wav'

# Obtiene una lista de nombres de archivos que coinciden con el patrón
file_list = glob.glob(pattern)
# Imprime la cantidad de archivos encontrados
print("Número de archivos encontrados:", len(file_list))
# Imprime la lista de nombres de archivos encontrados
print("Archivos encontrados:")
for file_name in file_list:
    print(file_name)

# Detección de silencio en el audio
Si la suma del silencio del audio es superior a 2.5 seg, no se considera el audio para su uso

In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_silence

def detectar_silencio(ruta_audio, umbral_silencio=-40, duracion_minima_silencio=500):
    # Cargar el archivo de audio
    audio = AudioSegment.from_file(ruta_audio)

    # Detectar los fragmentos de silencio
    fragmentos_silencio = detect_silence(audio, silence_thresh=umbral_silencio, seek_step=duracion_minima_silencio)

    if not fragmentos_silencio:
        print("El audio no contiene silencio.")
    else:
        print(f"Se detectaron fragmentos de silencio en {ruta_audio}:")
        sum = 0;
        for inicio, fin in fragmentos_silencio:
            print(f"    Inicio: {inicio}, Fin: {fin}")
            sum = sum+(fin-inicio)
            print(f"    Total time: {sum}")
        if  sum > 2500:
          print("audio no util")

In [ ]:
i=0
for file_name in file_list:
    gen_spect(file_name,i)
    detectar_silencio(file_name)
    i=i+1